In [2]:
import os
import json
from six import string_types
from translating_trees import Node
from translating_trees import pretty_print_tree
from translating_trees import encode_program
from translating_trees import make_tree_lambda
from translating_trees import make_tree_javascript
import json
from program_datasets import * 

In [ ]:
def figure_out_csharp(json_lists):
#     print(len(json_lists))
    all_tokens = set()
    children = {}
    parents = {}
    max_vars = 0
    max_consts = 0
    max_chars = 0
    max_strings = 0
    
    def tally_tokens(prog, parent=None):
        all_vars = set()
        all_consts = set()
        all_chars = set()
        all_strings = set()
        
        
        
        # ugly special cases
        if parent == "ArgumentList" and type(prog) == type("") and prog[0] == "(" and prog[-1] == ")":
            root_val = "()"
          
        if type(prog) == type("") or type(prog) == type(1): #TODO: is this second case necesary?
            # Add to parents
            if prog in parents:
                parents[prog].add(parent)
            else:
                parents[prog] = set([parent])
                
            return all_vars, all_consts, all_chars, all_strings
        
        for key in prog.keys():
             
            root_val = key
            
            if not type(root_val) == type(""):
                print("AAAAAAA")

            all_tokens.add(root_val)
            # Add to parents
            if root_val in parents:
                parents[root_val].add(parent)
            else:
                parents[root_val] = set([parent])
            # Add to children
            if parent in children:
                children[parent].add(root_val)
            else:
                children[parent] = set([root_val])
                
            if root_val == "IdentifierName":
                all_vars.add(prog[key])
            elif root_val == "NumericLiteralExpression":
                all_consts.add(prog[key])
            elif root_val == "CharacterLiteralExpression":
                all_chars.add(prog[key])
            elif root_val == "StringLiteralExpression":
                all_strings.add(prog[key])
            elif root_val == "VariableDeclarator" and type(prog[key]) == type(""):
                all_vars.add(prog[key])
            else:
                child_keys = []
                
                if type(prog[key]) == type([]):
                    for child in prog[key]:
                        for child_key in child.keys():
                            child_keys.append((child_key, child))
                elif type(prog[key]) == type(""):
                    child_keys.append((prog[key], prog[key]))
                else:
                    print("something weird is going on", prog)
                
                # Add to children
                for c in child_keys:
                    child_key, child = c
                            
                    child_vars, child_consts, child_chars, child_strings = tally_tokens(child, root_val)
                    all_vars |= child_vars
                    all_consts |= child_consts
                    all_chars |= child_chars
                    all_strings |= child_strings
                    
            return all_vars, all_consts, all_chars, all_strings
    
    i = 0
    for json_list in json_lists:
        print("new list")
        for prog in json_list:
            i += 1
            all_vars, all_consts, all_chars, all_strings = tally_tokens(prog)
            max_vars = max(max_vars, len(all_vars))
            max_consts = max(max_consts, len(all_consts))
            max_chars = max(max_chars, len(all_chars))
            max_strings = max(max_strings, len(all_strings))
#             if len(all_strings) > 1000:
#                 print("UH OH ON", i)
#                 print(all_consts)
#                 print(all_strings)
#                 print("======")
#                 print(prog)
                
#     print(all_tokens)
#     print(children)
#     print(parents)
#     print(max_vars)
#     print(max_consts)
#     print(max_chars)
#     print(max_strings)
    return all_tokens, children, parents, max_vars, max_consts, max_chars, max_strings

In [ ]:
# all_tokens, children, parents, max_vars, max_consts, max_chars, max_strings = figure_out_csharp(json_lists)

In [ ]:
def get_files(folder, suffix):
    matching_files = []
    for path, dirs, files in os.walk(folder):
        for name in files:
            if name[-len(suffix):] == suffix:
                matching_files.append(name)
    return matching_files

In [ ]:
def haskell_voodoo():
    files = get_files("filtered_data", "cs.txt")
    os.system("mkdir jsonified")
    os.system("ghc CleanCSharpData.hs -main-is CleanCSharpData.main -o CleanCSharpData;")
    for file in files:
        os.system("./CleanCSharpData filtered_data/{} > jsonified/{}".format(file, file))

In [ ]:
# Get all the C# datasets
def load_data(folder, extension):
    all_files = get_files(folder, extension)

    i = 0
    total_count = 0
    correct_count = 0

    json_lists = []
    for file in all_files:
        print("processing file", file)
        j_list = []
        with open(folder + "/" + file) as f:
            lines = f.readlines()
            print(len(lines))
            for line in lines:
                total_count += 1
                try:
                    prog = json.loads(line)
                    correct_count += 1
                    if not prog == []:
                        j_list.append(prog)
                except:
                    pass
                i += 1
        json_lists.append(j_list)
            
    print("TOTAL:", total_count)
    print("CORRECT:", correct_count)
    print(correct_count/total_count)
    return json_lists

In [ ]:
# haskell_voodoo()
# all_tokens, children, parents, max_vars, max_consts, max_chars, max_strings = figure_out_csharp(json_lists)

In [ ]:
# Blatantly copied from ProgramMatching.py
def isMatch(jprog, cprog):
    if not (len(jprog) == 3 and len(cprog) == 3):
        return False
    if jprog[2] == "[]" or cprog[2] == "[]":
        return False
    try:
        jName = jprog[1].replace('_','').lower()
        cName = cprog[1].replace('_','').lower()
        jFile = jprog[0][:jprog[0].index('.')].lower()
        if "/" in jFile:
            jFile = jFile.split("/")[-1]
        cFile = cprog[0][:cprog[0].index('.')].lower()
        if "/" in cFile:
            cFile = cFile.split("/")[-1]
        return (jName == cName) and (jFile == cFile)
    except:
        print("======JJJJJJJ======")
        print(jprog)
        print("=======CCCCCCC========")
        print(cprog)
        return "weirdness"

def filter_files(jfile, cfile, orig_folder, save_folder):
    d = {}
    
    jlist = []
    clist = []
    with open(orig_folder + "/" + jfile) as f:
        jlist = [line.split("\t") for line in f.readlines()]
        
    with open(orig_folder + "/" + cfile) as f:
        clist = [line.split("\t") for line in f.readlines()]
        
    i = 0
    for jprog in jlist:
        if not i%1000:
            print("i is ", i)
        i+=1
        for cprog in clist:
            match = isMatch(jprog,cprog)
            if match == "weirdness":
                return
            elif match:
                d[jprog[2]] = cprog[2]
    all_files = [(k,v) for k,v in d.items()]
    print("ok", len(all_files))
    # Write java
    with open(save_folder + "/" + jfile, "w") as f:
        for file in all_files:
            f.write(file[0])
    
    # Write C#
    with open(save_folder + "/" + cfile, "w") as f:
        for file in all_files:
            f.write(file[1])
            
        
    

In [ ]:
def filter():
    csharp_files = sorted(get_files("j2c-raw-data", "cs.txt"))
    java_files = sorted(get_files("j2c-raw-data", "java.txt"))[3:]

    print(csharp_files)
    print(java_files)

    for i in range(len(csharp_files)):
        print("file", java_files[i])
        filter_files(java_files[i], csharp_files[i], "j2c-raw-data", "filtered_data")

In [ ]:
def make_tree_java(json, long_base_case=True):
    if general_base_cases(json) is not None:
        return general_base_cases(json)
    
    # hopefully just one
    for key in json.keys():
        
        # I could put angle brackets by why bother?
        tag = key.upper()
        if not long_base_case:
            if tag == "identifier" or tag == "value":
                return make_tree_csharp(json[key])
        else:
            parentNode = Node(tag)
            children = json[key]
            if type(children) is list:
                parentNode.children.extend(map(lambda child: 
                                       make_tree_csharp(child, long_base_case=long_base_case), 
                                       children))
            else:
                parentNode.children.append(make_tree_csharp(children, long_base_case=long_base_case))
    return parentNode 

def canonicalize_java(tree):
    var_names = {}
    value_names = {} # could be string, int, char, bool
    
    def make_generic(node, dict, symbol):
        if node.value in dict:
            node.value = dict[node.value]
        else:
            new_symbol = symbol + str(len(dict) + 1)
            dict[node.value] = new_symbol
            node.value = new_symbol

    
    def canonicalize_java_helper(tree):
        if tree.value == "IDENTIFIER":
            make_generic(tree.children[0], num_names, "a")
        elif tree.value == "VALUE":
            make_generic(tree.children[0], var_names, "v")
        else:
            for child in tree.children:
                canonicalize_java_helper(child)
                
    canonicalize_java_helper(tree)
    return tree


"arguments":[{"name":{"identifier":"bytes"}},{"name":{"identifier":"length"},"scope":{"name":{"identifier":"bytes"}}}]


In [ ]:
csharp_json_lists = load_data("jsonified", "cs.txt")
java_json_lists = load_data("filtered_data", "java.txt")

In [ ]:
# For the moment, ignore the itext dataset
file_indices = [0, 1, 3, 4, 5] 

for index in file_indices:
    for i in range(5):
        print("C#")
        prog = csharp_json_lists[index][i]
        c_tree = make_tree_csharp(prog)
        better_c_tree = canonicalize_csharp(c_tree)
        pretty_print_tree(better_c_tree)
        print("JAVA")
        prog = java_json_lists[index][i]
        j_tree = make_tree_java(prog)
        better_c_tree = canonicalize_csharp(j_tree)
        pretty_print_tree(better_j_tree)
        


In [4]:
dset_train = ForLambdaDataset("../ANC/MainProgramDatasets/ForLambda/training_For.json",
                                       binarize_input=True, binarize_output=False, 
                                       eos_token=True, one_hot=False,
                                       num_ints=10, num_vars=11,
                                       long_base_case=True, 
                                       input_as_seq=False, 
                                       output_as_seq=False)
print("total length", len(dset_train))
for_counts = [node[0].size() for node in dset_train]
lambda_counts = [node[1].size() for node in dset_train]
print("for mean", sum(for_counts)/len(for_counts))
print("for max", max(for_counts))
print("for min", min(for_counts))
print("lambda mean", sum(lambda_counts)/len(lambda_counts))
print("lambda max", max(lambda_counts))
print("lambda min", min(lambda_counts))




total length 100000
for mean 22.01238
for max 104
for min 5
lambda mean 111.0692
lambda max 597
lambda min 25


In [7]:
# Load json file
js_progs = []
with open("../ANC/MainProgramDatasets/ForLambda/training_For.json") as f:
    json_all = f.read()
    print(json_all)
    json_progs = json.loads(json_all)
    for prog in json_progs:
        js_progs.append(prog)
print(len(js_progs))
nodes = [make_tree_FOR(prog, long_base_case=True) for prog in js_progs]




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100000


NameError: name 'make_tree_FOR' is not defined

In [8]:
_ = [print(json.dumps(x, indent=1)) for x in js_progs[:10]]

{
 "contents": [
  {
   "contents": [
    {
     "contents": [
      "a1",
      {
       "contents": [
        {
         "contents": 6,
         "tag": "Const"
        },
        {
         "contents": 2,
         "tag": "Const"
        }
       ],
       "tag": "Plus"
      }
     ],
     "tag": "Assign"
    },
    {
     "contents": [
      {
       "contents": [
        {
         "contents": 6,
         "tag": "Const"
        },
        {
         "contents": "a0",
         "tag": "Var"
        }
       ],
       "tag": "Ge"
      },
      {
       "contents": [
        "a0",
        {
         "contents": "a1",
         "tag": "Var"
        }
       ],
       "tag": "Assign"
      },
      {
       "contents": [
        "a2",
        {
         "contents": [
          {
           "contents": "a0",
           "tag": "Var"
          },
          {
           "contents": "a1",
           "tag": "Var"
          }
         ],
         "tag": "Minus"
        }
       ],
       "tag":

In [6]:
js_progs_new = [make_tree_javascript(prog, long_base_case=True) for prog in js_progs]


In [7]:
all_tokens = set()
def count_tokens(tree):
    all_tokens.add(tree.value)
    for child in tree.children:
        count_tokens(child)
    
    
    
def check_all():
    i = 0
    # Check grammar is right
    for prog in js_progs_new:
        count_tokens(prog)

check_all()
print(all_tokens)

{0, '<IFSTATEMENT>', 1, 3, 4, 5, 2, 7, 8, 9, 6, '<VARIABLEDECLARATOR>', '<ASSIGNMENTEXPRESSION>', '<LITERAL>', '+', 'a4', 'results2', 'call', '<IDENTIFIER>', '<PROGRAM>', '<RETURNSTATEMENT>', '<VARIABLEDECLARATION>', '<THISEXPRESSION>', 'results3', '<FUNCTIONEXPRESSION>', '===', '*', '<ARRAYEXPRESSION>', 'a1', 'a3', 'results', 'a0', '<BLOCKSTATEMENT>', '<EXPRESSIONSTATEMENT>', '=', '<MEMBEREXPRESSION>', 'a2', '<CALLEXPRESSION>', 'results1', '<WHILESTATEMENT>', 'push', 'a5'}


In [ ]:
# Function sto check grammars for consistency with datasets

d = {}
def check_extra(node, parent, child_index):
    if not (parent, child_index) in d:
        d[(parent, child_index)] = set([int(node.value)])
    else:
        d[(parent, child_index)].add(int(node.value))
    for i in range(len(node.children)):
        child = node.children[i]
        check_extra(child, int(node.value), i)



def check_valid(node, parent, child_index):
    if parent != -1:
        try:
            category = parent_to_category_coffee(num_vars, num_ints, parent)[child_index]
        except:
            print("Parent has too many kids!", parent, child_index, int(node.value))
            print(parent_to_category_coffee(num_vars, num_ints, parent))
            return False
        possible_outputs = category_to_child_coffee(num_vars, num_ints, category)
        if not int(node.value) in possible_outputs:
            print("ERROR!!!!!")
            print("parent", parent, " at index", child_index)
            print("had options", possible_outputs)
            print("in category", category)
            print("But got", int(node.value))
            return False
    for i in range(len(node.children)):
        if not check_valid(node.children[i], int(node.value), i):
            return False
    return True





for key in d.keys():
    if key[0] == -1:
        print(key)
    else:
        category = parent_to_category_coffee(num_vars, num_ints, key[0])[key[1]]
        possibilities = set(category_to_child_coffee(num_vars, num_ints, category))
        if possibilities != d[key]:
            print("parent: ", key[0])
            print('index', key[1])
            print('possibilities real', d[key])
            print("possibilities ours", possibilities)

# Check all the programs in a dataset for syntactic accuracy
# (this is a debugging function used to double check the accuracy of your grammar)
def check_all():
    i = 0
    # Check grammar is right
    for prog in numeric_progs:
        correct = check_extra(prog, -1, 0) # Or replace check_extra with check_valid
        if correct is False:
            print(i)
            pretty_print_tree(prog)
            return
        i += 1
        
check_all() #kangaroo
print(d)